In [1]:
import pandas as pd
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [2]:
train_data = pd.read_csv('house_datasets/train.csv')
test_data = pd.read_csv('house_datasets/test.csv')

In [3]:
train_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [4]:
test_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [5]:
# Шаг 1. добавил разграниение для тестовых и тренировочных данных
train_data['is_test'] = 0
test_data['is_test'] = 1

In [6]:
# Шаг 2. объединим данные
dataset = train_data.append(test_data, ignore_index=True)
dataset

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,is_test
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN,1
2915,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN,1
2916,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN,1
2917,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN,1


In [7]:
# Шаг 3. Изучим пропуски
columns = dict(dataset.isnull().sum())

with_bad_values = {}

for item in columns:
    if columns[item] != 0:
        with_bad_values[item] = columns[item]

with_bad_values

{'MSZoning': 4,
 'LotFrontage': 486,
 'Alley': 2721,
 'Utilities': 2,
 'Exterior1st': 1,
 'Exterior2nd': 1,
 'MasVnrType': 24,
 'MasVnrArea': 23,
 'BsmtQual': 81,
 'BsmtCond': 82,
 'BsmtExposure': 82,
 'BsmtFinType1': 79,
 'BsmtFinSF1': 1,
 'BsmtFinType2': 80,
 'BsmtFinSF2': 1,
 'BsmtUnfSF': 1,
 'TotalBsmtSF': 1,
 'Electrical': 1,
 'BsmtFullBath': 2,
 'BsmtHalfBath': 2,
 'KitchenQual': 1,
 'Functional': 2,
 'FireplaceQu': 1420,
 'GarageType': 157,
 'GarageYrBlt': 159,
 'GarageFinish': 159,
 'GarageCars': 1,
 'GarageArea': 1,
 'GarageQual': 159,
 'GarageCond': 159,
 'PoolQC': 2909,
 'Fence': 2348,
 'MiscFeature': 2814,
 'SaleType': 1,
 'SalePrice': 1459}

In [8]:
# хотя некоторые параметры дают почти 100% пропуск, 
# например PoolQC выкидывать их нельзя:
# отсутствие значения означает не пропуск, а отсутсвие бассейна

In [9]:
# Шаг 4. Пропущенные числовые значения заменим на медианы, а категориальные значения на слово EMPTY
number_columns = [name for name in dataset.columns if dataset[name].dtype in ['int64', 'float64']]
objects_columns = [name for name in dataset.columns if dataset[name].dtype == "object"]

values = {}

for item in objects_columns:
    values[item] = 'UNKNOWN'

for item in number_columns:
    values[item] = dataset[item].median()
        
dataset.fillna(value=values, inplace=True)   
dataset

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,is_test
0,1,60,RL,65.0,8450,Pave,UNKNOWN,Reg,Lvl,AllPub,...,UNKNOWN,UNKNOWN,UNKNOWN,0,2,2008,WD,Normal,208500.0,0
1,2,20,RL,80.0,9600,Pave,UNKNOWN,Reg,Lvl,AllPub,...,UNKNOWN,UNKNOWN,UNKNOWN,0,5,2007,WD,Normal,181500.0,0
2,3,60,RL,68.0,11250,Pave,UNKNOWN,IR1,Lvl,AllPub,...,UNKNOWN,UNKNOWN,UNKNOWN,0,9,2008,WD,Normal,223500.0,0
3,4,70,RL,60.0,9550,Pave,UNKNOWN,IR1,Lvl,AllPub,...,UNKNOWN,UNKNOWN,UNKNOWN,0,2,2006,WD,Abnorml,140000.0,0
4,5,60,RL,84.0,14260,Pave,UNKNOWN,IR1,Lvl,AllPub,...,UNKNOWN,UNKNOWN,UNKNOWN,0,12,2008,WD,Normal,250000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21.0,1936,Pave,UNKNOWN,Reg,Lvl,AllPub,...,UNKNOWN,UNKNOWN,UNKNOWN,0,6,2006,WD,Normal,163000.0,1
2915,2916,160,RM,21.0,1894,Pave,UNKNOWN,Reg,Lvl,AllPub,...,UNKNOWN,UNKNOWN,UNKNOWN,0,4,2006,WD,Abnorml,163000.0,1
2916,2917,20,RL,160.0,20000,Pave,UNKNOWN,Reg,Lvl,AllPub,...,UNKNOWN,UNKNOWN,UNKNOWN,0,9,2006,WD,Abnorml,163000.0,1
2917,2918,85,RL,62.0,10441,Pave,UNKNOWN,Reg,Lvl,AllPub,...,UNKNOWN,MnPrv,Shed,700,7,2006,WD,Normal,163000.0,1


In [10]:
# Шаг 5. Приведем к числовым значениям
dataset = pd.get_dummies(dataset)

In [11]:
# Шаг 6. Разделим данные
train_data = dataset.query("is_test == 0")
test_data = dataset.query("is_test == 1")

train_data = train_data.drop(['is_test'], axis=1)
test_data = test_data.drop(['is_test'], axis=1)

In [12]:
# Шаг 7. Подготовим данные 

#для обучения
x = train_data.drop('SalePrice', axis=1).values 
y = train_data["SalePrice"].values
y = y.astype("int")

# для теста
x_test = test_data.drop('SalePrice', axis=1).values

In [13]:
# Шаг 7.1 Обучим рандомный лес
random_forest_cv = RandomForestRegressor(n_estimators = 30, random_state=0)
random_forest_cv.fit(x, y)
y_random_forest = random_forest_cv.predict(x_test)
random_forest_cv.score(x, y)

0.9785383935676815

In [14]:
# Шаг 7.2. Обучим Ridge - регрессию
redge_cv = RidgeCV(alphas = (0.001, 0.01, 0.05, 0.1, 0.3, 0.5, 1, 3, 5, 10))
redge_cv.fit(x, y)
y_redge = redge_cv.predict(x_test)
redge_cv.score(x, y)

0.9012387395429758

In [15]:
# Шаг 7.3. XGB - регрессию
xgb_cv = xgb.XGBRegressor(n_estimators=340, max_depth=2, learning_rate=0.2)
xgb_cv.fit(x, y)
y_xgb = xgb_cv.predict(x_test)
xgb_cv.score(x, y)

0.9798134566337557

In [17]:
# Шаг 8. Усредним результат
y_result = (y_random_forest + y_redge + y_xgb)/3

result_for_csv = pd.DataFrame({
    'Id': test_data.Id.values, 
    'SalePrice': y_result
})

result_for_csv.to_csv('house_done.csv', index=False)